<a href="https://colab.research.google.com/github/Pasq98/Tesi/blob/main/Secondo%26TerzoEsperimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Caricamento librerie
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import tensorflow
from gensim.models.keyedvectors import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("gensim_glove_vectors.txt", binary=False)
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import os
os.environ['KERAS_BACKEND']='theano'
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import initializers, optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding, Dropout, LSTM, GRU, Bidirectional
from tensorflow.keras.models import Sequential

In [ ]:
#@title Caricamento dataset
def load_Cybertroll(): #Carico il dataset e ritorno il testo e i label
  CBT = pd.read_json('Dataset for Detection of Cyber-Trolls.json', lines= True)
  CBT["label"] = CBT.annotation.apply(lambda x: x.get('label'))
  CBT["label"] = CBT.label.apply(lambda x: x[0])
  remove_n = 1
  drop_indices = np.random.choice(CBT.index, remove_n, replace=False)
  CBT = CBT.drop(drop_indices)

  CBT = CBT.drop(CBT.query("label == '0'").sample(frac=.39).index)#elimino il 39% del dataset per bilanciarlo in fase di test
  print(CBT['label'].value_counts())

  x = CBT['content']
  label = CBT['label']
  print(x.shape)
  print(label.shape)

  return x, label


def load_Tweets():
  tweets=pd.read_csv('Tweets.csv')
  tweets_df=tweets.drop(tweets[tweets['airline_sentiment_confidence']<0.5].index,axis=0)
  tweets_df['text'] = tweets_df['text'].str.replace('@AmericanAir', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@USAirways', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@SouthwestAir', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@JetBlue', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@VirginAmerica', '')#Rimuovo hastag iniziali
  tweets_df['text'] = tweets_df['text'].str.replace('@united', '')#Rimuovo hastag iniziali

  #Rimuovo tweet con sentimento neutro
  tweets_df = tweets_df.loc[tweets_df["airline_sentiment"] != 'neutral']
  tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].str.replace('positive', '0')#Scambio etichette
  tweets_df['airline_sentiment'] = tweets_df['airline_sentiment'].str.replace('negative', '1')

  tweets_df = tweets_df.drop(tweets_df.query("airline_sentiment == '1'").sample(frac=.65).index)#elimino il 65% del dataset per bilanciarlo in fase di test
  print(tweets_df['airline_sentiment'].value_counts())

  x=tweets_df['text'] 
  label=tweets_df['airline_sentiment']

  return x, label

def load_sarcasm():
  data = pd.read_csv('Sarcasm.csv', sep=';')
  data['Column3'] =data['Column3'].str.replace('#sarcasm', '')
  data['Column3'] = data['Column3'].str.replace('[^\w\s#@/:%.,_-]', '', flags=re.UNICODE)

  x = data["Column3"]
  label = data['Column2']

  return x,label

def load_ACLL():
  acll=pd.read_csv('ACLLTrain.csv')
  #inverto le etichette
  acll['Review']= acll['Review'].str.replace('1','positive') #Scambio etichette per renderle coerenti con Cybertroll
  acll['Review']= acll['Review'].str.replace('0','negative')

  acll['Review']= acll['Review'].str.replace('positive','0')
  acll['Review']= acll['Review'].str.replace('negative','1')

  x = acll['Review']
  label = acll['Rating']
  return x, label


In [ ]:
#@title Standard preprocessing
def standard_preprocessing(text):
  clean_text = text.str.lower()
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  # Remove trailing spaces
  clean_text = clean_text.str.strip()
  # Remove Punctuations
  clean_text = clean_text.str.replace('[^\w\s]',' ')
  # Remove <br>
  clean_text = clean_text.str.replace('br', '')
  # Remove extra space in between words
  clean_text = clean_text.str.replace(' +', ' ')
  clean_text = clean_text.str.replace('\n', ' ')# per pad sequence
  # Remove Numbers
  clean_text = clean_text.str.replace('\d+', '')
  #stop word remove
  stop = stopwords.words('english')
  clean_text = clean_text.apply(lambda x: " ".join(x for x in x.split() if x not in stop ))
  #token
  #clean_text = clean_text.apply(nltk.word_tokenize) #NON FARLO CON EMBEDDING
  return clean_text

In [ ]:
#@title BLSTM model
def blstm(inp_dim, vocab_size, embed_size,emb_matri):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_size, weights=[emb_matri] ,input_length=inp_dim, trainable=True))
    model.add(Dropout(0.5))
    model.add(Bidirectional(LSTM(50)))
    model.add(Dropout(0.50))
    model.add(Dense(1, activation='sigmoid'))
    opt = optimizers.Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'],)

    return model

In [ ]:
#@title Get embedded matrix
def get_embedded_matrix(size_vocab):
  # create a weight matrix for words in training docs
  embedding_matrix = np.zeros((size_vocab, 50))

  for word, i in tokenizer.word_index.items():
    try:
      embedding_vector = glove_model.get_vector(word)
      embedding_matrix[i] = embedding_vector
    except KeyError:
      embedding_vector = 0
      embedding_matrix[i] = embedding_vector
  return embedding_matrix

In [ ]:
#@title Test Cross Dataset
#Tokenize the sentences
tokenizer = Tokenizer()

################ TEST DATASET
xTEST, labelTEST = load_Cybertroll()
xTEST = standard_preprocessing(xTEST)

#preparing vocabulary
tokenizer.fit_on_texts(list(xTEST))
MAX_LEN = 50
xTEST = tokenizer.texts_to_sequences(xTEST)
xTEST = pad_sequences(xTEST, MAX_LEN)#MAX LEN dovrebbe essere il numero di massimo di parole in una frase

labelTEST = [int(lab) for lab in labelTEST] #Necessario per piu dataset

size_of_vocabularyTEST=len(tokenizer.word_index) + 1 #+1 for padding
print("Parole del dizionario: ",size_of_vocabularyTEST)

embedding_matrixTEST = get_embedded_matrix(size_of_vocabularyTEST)

#TRAIN DATASET
xTrain, labelTrain = load_ACLL()
xTrain = standard_preprocessing(xTrain)

#preparing vocabulary
tokenizer.fit_on_texts(list(xTrain))
MAX_LEN = 50
xTrain = tokenizer.texts_to_sequences(xTrain)
xTrain = pad_sequences(xTrain, MAX_LEN)#MAX LEN dovrebbe essere il numero di massimo di parole in una frase

labelTrain = [int(lab) for lab in labelTrain] #Necessario per piu dataset

size_of_vocabularyTrain=len(tokenizer.word_index) + 1 #+1 for padding
print("Parole del dizionario: ",size_of_vocabularyTrain)

embedding_matrixTrain = get_embedded_matrix(size_of_vocabularyTrain)


X_trainTEST, X_testTEST, y_trainTEST, y_testTEST = train_test_split(xTEST, labelTEST, test_size = 0.2, random_state = 1)

#physical_devices = tensorflow.config.experimental.list_physical_devices('GPU') #Direttive per il server del dipartimento
#tensorflow.config.experimental.set_memory_growth(physical_devices[0], True)


model = blstm(MAX_LEN,size_of_vocabularyTrain,50,embedding_matrixTrain)
model.fit(np.array(xTrain),np.array(labelTrain), epochs = 6, batch_size=64, validation_split=0.4)#val_split 0.4 for train ACLL e Tweets
y_pred = model.predict(X_testTEST)                                                                #0.2 other
y_pred = (y_pred > 0.5)
report = classification_report(y_testTEST, y_pred)
print(report)
